# Prompting e Output Estruturado

A qualidade das respostas de um modelo de linguagem depende diretamente da qualidade das instrucoes que ele recebe. Esse processo de formular instrucoes e chamado de **prompting**.

Neste notebook vamos explorar diferentes tecnicas de prompting, desde instrucoes simples ate a extracao de dados estruturados com validacao de tipos usando Pydantic.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Prompting basico

Quando criamos um agente sem nenhuma instrucao adicional, ele se comporta como um assistente generico. A resposta vai depender exclusivamente do treinamento do modelo.

In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agente = create_agent(model="gpt-4.1-nano")

In [ ]:
pergunta = HumanMessage(content="Analise a empresa Nubank.")

resposta = agente.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

A resposta e generica porque o modelo nao sabe qual o papel dele na conversa. Podemos mudar completamente o comportamento do agente adicionando um **system prompt**, que define a persona e as instrucoes que o modelo deve seguir.

In [ ]:
system_prompt = "Voce e um analista de negocios especializado em startups brasileiras. Seja direto e objetivo nas analises."

analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt=system_prompt
)

In [ ]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

Observe como a mesma pergunta gerou uma resposta completamente diferente. O system prompt e a forma mais direta de controlar o comportamento do modelo.

## Few-shot prompting

Em muitos casos, descrever o que voce quer nao e suficiente. E mais eficaz **mostrar exemplos** do formato esperado. Essa tecnica e chamada de **few-shot prompting**: incluimos alguns pares de entrada e saida no system prompt para que o modelo aprenda o padrao desejado.

In [ ]:
system_prompt = """
Voce e um analista de negocios especializado em startups brasileiras.
Responda com analises curtas e diretas.

Usuario: Analise a empresa iFood.
Analista: iFood domina o delivery de comida no Brasil com mais de 80% de market share. Modelo de marketplace com receita via comissoes e assinaturas. Desafio principal: manter margens com alto custo de logistica.

Usuario: Analise a empresa QuintoAndar.
Analista: QuintoAndar revolucionou o aluguel de imoveis ao eliminar fiador e burocracias. Modelo de plataforma com receita via taxas de intermediacao. Expandiu para compra e venda, aumentando o mercado enderecavel.
"""

analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt=system_prompt
)

In [ ]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

Com os exemplos, o modelo entendeu o **tom**, o **tamanho** e a **estrutura** que queremos. Isso e muito mais eficiente do que tentar descrever o formato com palavras.

## Prompt estruturado

Outra abordagem e definir explicitamente os **campos** que a resposta deve conter. Isso funciona como um template que o modelo preenche. E util quando voce precisa de consistencia entre multiplas chamadas.

In [ ]:
system_prompt = """
Voce e um analista de negocios especializado em startups brasileiras.

Para cada empresa analisada, responda exatamente no formato abaixo:

Empresa: nome da empresa
Setor: setor de atuacao
Modelo de negocio: como a empresa gera receita
Diferencial: principal vantagem competitiva
Risco: principal risco ou desafio
"""

analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt=system_prompt
)

In [ ]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

In [ ]:
resposta = analista.invoke(
    {"messages": [HumanMessage(content="Analise a empresa Stone.")]}
)

print(resposta["messages"][-1].content)

O formato e consistente entre chamadas diferentes, o que e importante para pipelines automatizados. Porem, a resposta ainda e texto puro. Se quisermos usar esses dados em codigo (salvar em banco, comparar empresas, gerar relatorios), precisamos de algo mais robusto.

## Structured output

O LangChain permite forcar o modelo a retornar dados em um formato **estruturado e tipado**, usando modelos do Pydantic. Em vez de receber texto livre, recebemos um objeto Python com campos validados.

Isso e fundamental para integrar LLMs em sistemas reais, onde os dados precisam ser consumidos por outras partes do codigo.

In [ ]:
from pydantic import BaseModel

class AnaliseEmpresa(BaseModel):
    empresa: str
    setor: str
    modelo_de_negocio: str
    diferencial: str
    risco: str

In [ ]:
analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt="Voce e um analista de negocios especializado em startups brasileiras.",
    response_format=AnaliseEmpresa
)

In [ ]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

resposta["structured_response"]

O retorno agora e um objeto `AnaliseEmpresa`, nao texto. Podemos acessar cada campo diretamente como atributo Python.

In [ ]:
analise = resposta["structured_response"]

print(type(analise))

In [ ]:
print(analise.empresa)
print(analise.setor)
print(analise.diferencial)

In [ ]:
print(f"{analise.empresa} atua no setor de {analise.setor}.")
print(f"Diferencial: {analise.diferencial}")
print(f"Risco: {analise.risco}")

Essa abordagem transforma o LLM em uma **funcao que retorna dados tipados**, o que permite integrar facilmente com bancos de dados, APIs, dashboards ou qualquer outro sistema. A validacao dos tipos e feita automaticamente pelo Pydantic, garantindo que os dados estejam no formato esperado.